In [12]:
import time
import torch
import faiss
import pathlib
from PIL import Image
import numpy as np
import pandas as pd
import os
import time
import cv2
from pathlib import Path

from skimage.feature import graycomatrix, graycoprops
from skimage.color import rgb2gray
from skimage.feature import hog
from skimage import io, color

import streamlit as st
from streamlit_cropper import st_cropper    

from tensorflow import keras
from tensorflow.keras import Model

from sklearn.feature_extraction.image import extract_patches_2d
from scipy.cluster.vq import kmeans, vq

from tensorflow.keras.applications import VGG19
from tensorflow.keras.applications.vgg19 import preprocess_input

os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"

st.set_page_config(layout="wide")

device = torch.device('cpu')

FILES_PATH = str(pathlib.Path().resolve())

# Path in which the images should be located
IMAGES_PATH = os.path.join(FILES_PATH, 'DatasetArteTrainTest/Train')
# Path in which the database should be located
DB_PATH = os.path.join(FILES_PATH, 'database')
DB_FILE = 'db.csv' # name of the database

def preprocess_image(image_path, target_size=(224, 224)):
    # Cargar la imagen
    img = Image.open(image_path).convert('RGB')
    # Redimensionar la imagen al tamaño objetivo
    img = img.resize(target_size)
    # Convertir la imagen en un array de numpy y normalizar (escalar píxeles a 0-1)
    img_array = np.array(img)
    return img_array

def get_image_list():
    
    df = pd.read_csv(os.path.join(DB_PATH, DB_FILE))
    image_list = list(df.image.values)

    return image_list

def calculate_histograms(image, bins=32):
    red = cv2.calcHist([image], [2], None, [bins], [0, 256])
    green = cv2.calcHist([image], [1], None, [bins], [0, 256])
    blue = cv2.calcHist([image], [0], None, [bins], [0, 256])
    vector = np.concatenate([red, green, blue], axis=0)
    vector = vector.reshape(-1)
    vector = np.array(vector)
    vector = vector.reshape(vector.shape[0], -1)
    vector = vector.reshape(1, -1)
    return vector

def calcular_histograma_textura(imagen, distancias=[5], angulos=[45]):
    """
    Calcula un histograma basado en propiedades de textura (GLCM).
    """
    imagen_gris = rgb2gray(imagen)
    imagen_gris = (imagen_gris * 255).astype(np.uint8)

    # Calcular GLCM
    glcm = graycomatrix(imagen_gris, distances=distancias, angles=angulos, symmetric=True, normed=True)
    
    # Propiedades de textura
    propiedades = ['contrast', 'dissimilarity', 'homogeneity', 'ASM', 'energy', 'correlation']
    valores_textura = {prop: graycoprops(glcm, prop).flatten() for prop in propiedades}
    
    # Crear un histograma concatenando los valores de textura
    histograma = np.concatenate([valores_textura[prop] for prop in propiedades])
    histograma = histograma.reshape(histograma.shape[0], -1)
    histograma = histograma.reshape(1, -1)
    return histograma

def extract_image_patches(image, random_state, patch_size=(30, 30), n_patches=250):
    patches = extract_patches_2d(
        image, 
        patch_size=patch_size, 
        max_patches=n_patches, 
        random_state=random_state
    )
    return patches.reshape((n_patches, -1))

def bag_of_words_extractor(img_query):
    codebook = np.load('Bag_Of_Words.npy')
    orb = cv2.ORB_create()
    image = np.asarray(img_query)
    patch_size=(30, 30)
    n_patches=250
    random_seed=0
    random_state = np.random.RandomState(random_seed)
    patches = []
    patches.append(extract_image_patches(image, random_state, patch_size, n_patches))

    keypoints = []
    descriptors = []
    for patch in patches:
        patch_keypoints, patch_descriptors = orb.detectAndCompute(patch, None)
        keypoints.append(patch_keypoints)
        descriptors.append(patch_descriptors)

    visual_words = [vq(desc, codebook)[0] for desc in descriptors]
    frequency_values = [np.bincount(word, minlength=250) for word in visual_words]
    frequency_values = np.float32(frequency_values)
    faiss.normalize_L2(frequency_values)

    return frequency_values

def compute_deep_features(image):
    # Carga el modelo preentrenado VGG19
    base_model = VGG19(weights='imagenet', include_top=False)
    model = Model(inputs=base_model.input, outputs=base_model.layers[-2].output)
    # Ajusta la imagen al tamaño esperado por el modelo
    image_resized = cv2.resize(image, (224, 224))
    image_array = np.expand_dims(image_resized, axis=0)
    image_preprocessed = preprocess_input(image_array)
    # Extrae características
    features = model.predict(image_preprocessed)
    features = features.flatten()
    return features.reshape(1, -1)

def autoencoder_extractor(image: np.array, encoder):
    # batch dimension
    image = np.expand_dims(image, axis=0)
    feature_vector = encoder.predict(image)
    feature_vector = feature_vector.reshape(1, -1)
    return feature_vector

def retrieve_image(img_query, feature_extractor, n_imgs=11):

    if (feature_extractor == 'Extractor 1: Histograma Color'):
        model_feature_extractor = calculate_histograms(img_query)
        indexer = faiss.read_index(os.path.join(DB_PATH,  'feat_extract_1.index'))
        if model_feature_extractor is None:
            return None

    elif (feature_extractor == 'Extractor 2: Texturas'):
        model_feature_extractor = calcular_histograma_textura(img_query)
        indexer = faiss.read_index(os.path.join(DB_PATH,  'feat_extract_2.index'))
        if model_feature_extractor is None:
            return None

    elif (feature_extractor == 'Extractor 3: Bag Of Words'):
        model_feature_extractor = bag_of_words_extractor(img_query)
        indexer = faiss.read_index(os.path.join(DB_PATH,  'feat_extract_3.index'))
        if model_feature_extractor is None:
            return None

    elif (feature_extractor == 'Extractor 4: CNN-VGG19'):
        img_query = np.array(img_query) / 255.0
        model_feature_extractor = compute_deep_features(img_query)
        indexer = faiss.read_index(os.path.join(DB_PATH,  'feat_extract_4.index'))
        if model_feature_extractor is None:
            return None
        
    elif (feature_extractor == 'Extractor 5: Autoencoder'):
        img_query = np.array(img_query) / 255.0
        filename = 'autoencoder.keras'
        autoencoder = keras.models.load_model(filename)
        encoder = Model(inputs=autoencoder.input, outputs=autoencoder.get_layer('dec_conv0').output)
        model_feature_extractor = autoencoder_extractor(img_query, encoder)
        indexer = faiss.read_index(os.path.join(DB_PATH,  'feat_extract_5.index'))
        if model_feature_extractor is None:
            return None
    
    else:
        model_feature_extractor = calculate_histograms(img_query)
        indexer = faiss.read_index(os.path.join(DB_PATH,  'feat_extract_1.index'))
        if model_feature_extractor is None:
            return None

    # TODO: Modify accordingly

    _, indices = indexer.search(model_feature_extractor, k=n_imgs)

    return indices[0]

def metric_precision(img_query, retriev, k_values=[1, 3, 5, 7, 11]):
    img_query_name = img_query.split('-')[1].split('.')[0]   
    image_list = get_image_list()
    precision_values = {}

    for k in k_values:
        if k > len(retriev):
            continue  # Skip if k is not possible

        retrieved_image_names = [image_list[retriev[i]].split('-')[1].split('.')[0] for i in range(k)]
        # Count number of images with same label. 
        relevant_count = sum(1 for img in retrieved_image_names if img == img_query_name)
        precision = relevant_count / k
        precision_values[k] = round(precision, 2)
        
    return precision_values

2024-12-03 17:36:51.777 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


In [31]:
def calculate_results(): 
    extractores = ['Extractor 1: Histograma Color', 'Extractor 2: Texturas', 'Extractor 3: Bag Of Words', 'Extractor 4: CNN-VGG19', 'Extractor 5: Autoencoder']
    df_columns = ['1_K1', '1_K3', '1_K5', '1_K7', '1_K11', '2_K1', '2_K3', '2_K5', '2_K7', '2_K11', '3_K1', '3_K3', '3_K5', '3_K7', '3_K11', '4_K1', '4_K3', '4_K5', 
                '4_K7', '4_K11', '5_K1', '5_K3', '5_K5', '5_K7', '5_K11', 'Movement']
    df = pd.DataFrame(columns=df_columns)
    test_dir = "./DatasetArteTrainTest/Test/"
    n_test = len(os.listdir(test_dir))

    for i in range(n_test):
        query_index = i
        query_image_dir = Path(os.path.join(test_dir, os.listdir(test_dir)[query_index]))
        query_image = preprocess_image(query_image_dir)
        label = query_image_dir.name.split('-')[1].split('.')[0]
        df.loc[i, 'Movement'] = label
        for j in range(len(extractores)):
            extractor = extractores[j]
            similar_images = retrieve_image(query_image, extractor)
            precision_results = metric_precision(query_image_dir.name, similar_images)
            for k, precision in precision_results.items():
                column_name = str(j+1)+'_K'+str(k)
                df.loc[i, column_name] = precision
    return df

In [32]:
archivo_excel = 'ResultadosCBIR.xlsx'

if os.path.exists(archivo_excel):
    df = pd.read_excel(archivo_excel)
else:
    df = calculate_results()
    df.to_excel('ResultadosCBIR.xlsx')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 419ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 401ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 401ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 384ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 388ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 397ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 387ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 401ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 390ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 100ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 389ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 85ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 382ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 384ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 384ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/ste

In [35]:
df.head()

,1_K1,1_K3,1_K5,1_K7,1_K11,2_K1,2_K3,2_K5,2_K7,2_K11,...,4_K3,4_K5,4_K7,4_K11,5_K1,5_K3,5_K5,5_K7,5_K11,Movement
0,1.0,1.0,1.0,1.0,1.0,0.0,0.33,0.6,0.43,0.45,...,1.0,1.0,1.0,1.0,1.0,0.67,0.6,0.71,0.55,Rococo
1,1.0,1.0,1.0,1.0,1.0,0.0,0.33,0.2,0.43,0.27,...,1.0,0.8,0.71,0.55,0.0,0.33,0.6,0.71,0.73,Rococo
2,1.0,1.0,1.0,1.0,1.0,1.0,0.67,0.4,0.43,0.27,...,1.0,1.0,1.0,1.0,1.0,0.67,0.8,0.86,0.73,Rococo
3,1.0,1.0,1.0,1.0,1.0,1.0,0.33,0.4,0.57,0.64,...,1.0,1.0,1.0,1.0,1.0,1.0,0.8,0.86,0.64,Rococo
4,1.0,1.0,1.0,1.0,0.91,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.14,0.18,1.0,1.0,0.8,0.86,0.91,Baroque


In [88]:
columns_1 = [col for col in df.columns if '1_' in col]
means_1 = df[columns_1].mean()

columns_2 = [col for col in df.columns if '2_' in col]
means_2 = df[columns_2].mean()

columns_3 = [col for col in df.columns if '3_' in col]
means_3 = df[columns_3].mean()

columns_4 = [col for col in df.columns if '4_' in col]
means_4 = df[columns_4].mean()

columns_5 = [col for col in df.columns if '5_' in col]
means_5 = df[columns_5].mean()

k =[1, 3, 5, 7, 11]
for i in range(5):
    print('Histograma de Color para k = ', k[i], ': ', round(means_1.iloc[i], 2))
    print('Texturas para k = ', k[i], ': ', round(means_2.iloc[i], 2))
    print('Bag Of Words para k = ', k[i], ': ', round(means_3.iloc[i], 2))
    print('VGG19 para k = ', k[i], ': ', round(means_4.iloc[i], 2))
    print('Autoencoder para k = ', k[i], ': ', round(means_5.iloc[i], 2))

Histograma de Color para k =  1 :  0.69
Texturas para k =  1 :  0.25
Bag Of Words para k =  1 :  0.44
VGG19 para k =  1 :  0.56
Autoencoder para k =  1 :  0.86
Histograma de Color para k =  3 :  0.69
Texturas para k =  3 :  0.28
Bag Of Words para k =  3 :  0.47
VGG19 para k =  3 :  0.57
Autoencoder para k =  3 :  0.84
Histograma de Color para k =  5 :  0.64
Texturas para k =  5 :  0.28
Bag Of Words para k =  5 :  0.45
VGG19 para k =  5 :  0.54
Autoencoder para k =  5 :  0.84
Histograma de Color para k =  7 :  0.62
Texturas para k =  7 :  0.27
Bag Of Words para k =  7 :  0.43
VGG19 para k =  7 :  0.5
Autoencoder para k =  7 :  0.83
Histograma de Color para k =  11 :  0.61
Texturas para k =  11 :  0.25
Bag Of Words para k =  11 :  0.39
VGG19 para k =  11 :  0.47
Autoencoder para k =  11 :  0.78


In [89]:
print('Media Histograma de Color: ', round(means_1.mean(), 2))
print('Media Texturas: ', round(means_2.mean(), 2))
print('Media Bag Of Words: ', round(means_3.mean(), 2))
print('Media VGG19: ', round(means_4.mean(), 2))
print('Media Autoencoder: ', round(means_5.mean(), 2))

Media Histograma de Color:  0.65
Media Texturas:  0.27
Media Bag Of Words:  0.44
Media VGG19:  0.53
Media Autoencoder:  0.83


In [72]:
indices_rococo = list(df.loc[df['Movement'] == 'Rococo'].index)
indices_baroque = df.loc[df['Movement'] == 'Baroque'].index
indices_realism = df.loc[df['Movement'] == 'Realism'].index
indices_japanese = df.loc[df['Movement'] == 'Japanese_Art'].index
indices_art_nouveau = df.loc[df['Movement'] == 'Art_Nouveau'].index
indices_medieval = df.loc[df['Movement'] == 'Western_Medieval'].index
titulos_indices = ['Rococo', 'Baroque', 'Realism', 'Japanese_Art', 'Art_Nouveau', 'Western_Medieval']

extractor_1 = df[columns_1]
extractor_2 = df[columns_2]
extractor_3 = df[columns_3]
extractor_4 = df[columns_4]
extractor_5 = df[columns_5]

indices_movimientos = [indices_rococo, indices_baroque, indices_realism, indices_japanese, indices_art_nouveau, indices_medieval]
df_extractores = [extractor_1, extractor_2, extractor_3, extractor_4, extractor_5]

In [81]:
i = 0
for extractor in df_extractores:
    j = 0
    for movimiento in indices_movimientos:
        print('Media extractor ', i, ' en ', titulos_indices[j], ': ', round(extractor.loc[movimiento, :].mean().mean(), 2))
        j += 1
    i += 1

Media extractor  0  en  Rococo :  0.78
Media extractor  0  en  Baroque :  0.91
Media extractor  0  en  Realism :  0.64
Media extractor  0  en  Japanese_Art :  0.46
Media extractor  0  en  Art_Nouveau :  0.75
Media extractor  0  en  Western_Medieval :  0.35
Media extractor  1  en  Rococo :  0.29
Media extractor  1  en  Baroque :  0.15
Media extractor  1  en  Realism :  0.29
Media extractor  1  en  Japanese_Art :  0.3
Media extractor  1  en  Art_Nouveau :  0.27
Media extractor  1  en  Western_Medieval :  0.3
Media extractor  2  en  Rococo :  0.47
Media extractor  2  en  Baroque :  0.59
Media extractor  2  en  Realism :  0.33
Media extractor  2  en  Japanese_Art :  0.25
Media extractor  2  en  Art_Nouveau :  0.48
Media extractor  2  en  Western_Medieval :  0.5
Media extractor  3  en  Rococo :  0.67
Media extractor  3  en  Baroque :  0.16
Media extractor  3  en  Realism :  0.69
Media extractor  3  en  Japanese_Art :  0.4
Media extractor  3  en  Art_Nouveau :  0.64
Media extractor  3  en  W